# Notes for FUSE.init

In [1]:
using Revise
using FUSE
using IMAS
using IMASDD
using Printf
using Plots;
FUSE.logging(Logging.Info);

[ Info: Precompiling FUSE [e64856f0-3bb8-4376-b4b7-c03396503992]


## Load ini and act from FPP v1_demount case

In [2]:
ini, act = FUSE.case_parameters(:FPP, version=:v1_demount, init_from=:scalars);

## Set ini and act parameters

In [3]:
ini.equilibrium.ϵ = 1/3.5
ini.equilibrium.κ = 0.8826 # set less than one so that init uses this as the f_kappa parameter
ini.equilibrium.δ = 0.7
ini.equilibrium.ζ = 0.0
ini.equilibrium.pressure_core = 1e6

ini.build.plasma_gap = 0.2

ini.core_profiles.impurity = :Kr
ini.core_profiles.n_shaping = 2.5
ini.core_profiles.T_shaping = 2.5
ini.core_profiles.T_ratio = 0.825
ini.core_profiles.greenwald_fraction = 1.0
ini.core_profiles.ne_ped = missing
ini.core_profiles.greenwald_fraction_ped = 0.7
ini.core_profiles.helium_fraction = 0.04

ini.equilibrium.xpoints = :lower
ini.build.divertors = :lower
ini.build.symmetric = false

ini.ec_launchers.power_launched = 55e6

ini.pf_active.n_coils_outside = 8

ini.requirements.flattop_duration = 24*3600.0

86400.0

In [4]:
act.ActorEquilibriumTransport.do_plot = false
act.ActorEquilibriumTransport.max_iter = 1

act.ActorEquilibrium.model = :Solovev
act.ActorEquilibrium.symmetrize = false

act.ActorTauenn.transport_model = :ds03
act.ActorTauenn.confinement_factor = 0.9
act.ActorTauenn.T_shaping = 2.5
act.ActorTauenn.T_ratio_pedestal = 0.825
act.ActorTauenn.error = 1e-3
act.ActorTauenn.eped_only_powerlaw = true # more reliable for broader exploration
act.ActorTauenn.verbose = true

true

## Run FUSE.init

In [5]:
dd = IMAS.dd();
FUSE.init(dd, ini, act; do_plot=false);


## Scan over pressure_core

In [11]:
pressure_core_list = [0.1e6, 1.0e6, 2.0e6, 2.5e6]
dd_init_list = Vector{Any}([])
dd_eqtrans_list = Vector{Any}([])

for pressure_core in pressure_core_list
    ini0 = deepcopy(ini)
    act0 = deepcopy(act)

    ini0.equilibrium.pressure_core = pressure_core

    dd0 = IMAS.dd();

    FUSE.init(dd0, ini0, act0; do_plot=false);
    append!(dd_init_list,[dd0]);
    display("Success!")

    dd1 = deepcopy(dd0)

    try
        FUSE.ActorEquilibriumTransport(dd1, act0);
        append!(dd_eqtrans_list, [dd1])
    catch e
        @printf("ActorEquilibriumTransport failed for P0 = %.2e",pressure_core)
        display(e)
        append!(dd_eqtrans_list, [nothing])
    end

end

"Success!"

"Nelder-Mead"

"2.000 KeV ==> cost 1.65e+00"

"3.000 KeV ==> cost 1.58e+00"

"4.000 KeV ==> cost 1.51e+00"

...

"-0.003 KeV ==> cost 2.07e-03"

"-0.003 KeV ==> cost 2.07e-03"

 * Status: success

 * Candidate solution
    Final objective value:     2.065772e-03

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-03

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    9
    f(x) calls:    21

method = Optim.NelderMead{Optim.AffineSimplexer, Optim.AdaptiveParameters}(Optim.AffineSimplexer(0.025, 0.5), Optim.AdaptiveParameters(1.0, 1.0, 0.75, 1.0))
initial_x = [2000.0]
minimizer = [-3.4070175170900256]
minimum = 0.0020657718326202446
iterations = 9
iteration_converged = false
x_converged = false
x_abstol = 0.0
x_reltol = 0.0
x_abschange = NaN
x_relchange = NaN
f_converged = false
f_abstol = 0.0
f_reltol = 0.0
f_abschange = NaN
f_relchange = NaN
g_converged = true
g_abstol = 0.001
g_residual = 0.0001057287104392346
f_increased = false
f_calls = 21
g_calls = 0
h_calls = 0
ls_success = true
time_limit = NaN
time_run = 0.4957869052886963
stopped_by = (f_limit_reached = false, g_limit_reached = false,

error = 0.001
eped_factor = 1.0
rho_fluxmatch = 0.6
T_shaping = 2.5
T_ratio_pedestal = 0.825
transport_model = ds03
confinement_factor = 0.9
warn_nn_train_bounds = false
eped_only_powerlaw = true
update_pedestal = true


┌ Warning: Max number of iterations (1) has been reached with convergence error of [0.564] compared to threshold of 0.05
└ @ FUSE /fusion/ga/projects/ird/ptp/weisbergd/julia/dev/FUSE/src/actors/compound/equilibrium_transport_actor.jl:105


In [12]:
using Printf

for (i,p0) in enumerate(pressure_core_list)
    
    dd_init = dd_init_list[i]
    cs_init = dd_init.core_sources
    cp1d_init = dd_init.core_profiles.profiles_1d[]
    total_source_init = IMAS.total_sources(cs_init, cp1d_init; fields=[:power_inside, :total_ion_power_inside])

    dd_eqtr = dd_eqtrans_list[i]
    cs_eqtr = dd_eqtr.core_sources
    cp1d_eqtr = dd_eqtr.core_profiles.profiles_1d[]
    total_source_eqtr = IMAS.total_sources(cs_eqtr, cp1d_eqtr; fields=[:power_inside, :total_ion_power_inside])

    @printf("-------------------\n")
    @printf("P0_ini = %.2f MPa\n", p0/1e6)
    @printf("-------------------\n")

    @printf("Param\tUnit\tInit\tEqTrans\n")
    @printf("-----\t-----\t-----\t-----\n")
    @printf("P0mhd\tMPa\t%.2f\t%.2f\n",
        dd_init.equilibrium.time_slice[2].profiles_1d.pressure[1]/1e6,
        dd_eqtr.equilibrium.time_slice[2].profiles_1d.pressure[1]/1e6)

    @printf("P0tot\tMPa\t%.2f\t%.2f\n",
        cp1d_init.pressure[1]/1e6,
        cp1d_eqtr.pressure[1]/1e6)
    
    @printf("Te0\tkeV\t%.1f\t%.1f\n",
        cp1d_init.electrons.temperature[1]/1e3,
        cp1d_eqtr.electrons.temperature[1]/1e3)

    @printf("Pfus\tMW\t%.1f\t%.1f\n",
        IMAS.fusion_power(dd_init)/1e6,
        IMAS.fusion_power(dd_eqtr)/1e6)

    @printf("Pheat\tMW\t%.1f\t%.1f\n",
        (total_source_init.electrons.power_inside[end] + total_source_init.total_ion_power_inside[end] - IMAS.radiation_losses(cs_init))/1e6,
        (total_source_eqtr.electrons.power_inside[end] + total_source_eqtr.total_ion_power_inside[end] - IMAS.radiation_losses(cs_eqtr))/1e6,)

    @printf("Prad\tMW\t%.1f\t%.1f\n",
        IMAS.radiation_losses(cs_init)/1e6,
        IMAS.radiation_losses(cs_eqtr)/1e6)

    @printf("Wth\tMJ\t%.1f\t%.1f\n",
        IMAS.energy_thermal(cp1d_init)/1e6,
        IMAS.energy_thermal(cp1d_eqtr)/1e6)

    @printf("βn\t-\t%.1f\t%.1f\n",
        dd_init.summary.global_quantities.beta_tor_norm.value[1],
        dd_eqtr.summary.global_quantities.beta_tor_norm.value[1])

    @printf("τe_th\ts\t%.2f\t%.2f\n",
        IMAS.tau_e_thermal(cp1d_init, cs_init),
        IMAS.tau_e_thermal(cp1d_eqtr, cs_eqtr))

    @printf("τe_ds03\ts\t%.2f\t%.2f\n",
        IMAS.tau_e_ds03(dd_init),
        IMAS.tau_e_ds03(dd_eqtr))

    @printf("ds03\t-\t%.2f\t%.2f\n",
        IMAS.tau_e_thermal(cp1d_init, cs_init) / IMAS.tau_e_ds03(dd_init),
        IMAS.tau_e_thermal(cp1d_eqtr, cs_eqtr) / IMAS.tau_e_ds03(dd_eqtr))
    @printf("\n")
end



-------------------
P0_ini = 0.10 MPa
-------------------
Param	Unit	Init	EqTrans
-----	-----	-----	-----
P0mhd	MPa	0.10	2.78
P0tot	MPa	0.09	2.73
Te0	keV	2.0	66.0
Pfus	MW	0.1	1303.6
Pheat	MW	56.3	316.9
Prad	MW	-246.1	-86.7
Wth	MJ	21.5	271.7
βn	-	0.4	8.7
τe_th	s	0.38	0.86
τe_ds03	s	2.46	0.95
ds03	-	0.16	0.90

-------------------
P0_ini = 1.00 MPa
-------------------
Param	Unit	Init	EqTrans
-----	-----	-----	-----
P0mhd	MPa	1.00	1.67
P0tot	MPa	1.05	1.65
Te0	keV	20.3	33.0
Pfus	MW	349.3	758.1
Pheat	MW	125.0	207.1
Prad	MW	-67.8	-67.3
Wth	MJ	172.2	224.8
βn	-	3.4	5.1
τe_th	s	1.38	1.09
τe_ds03	s	1.59	1.21
ds03	-	0.86	0.90

-------------------
P0_ini = 2.00 MPa
-------------------
Param	Unit	Init	EqTrans
-----	-----	-----	-----
P0mhd	MPa	2.00	0.89
P0tot	MPa	2.40	0.88
Te0	keV	40.5	3.3
Pfus	MW	1057.4	3.4
Pheat	MW	267.2	55.7
Prad	MW	-69.0	-129.1
Wth	MJ	349.9	183.8
βn	-	7.6	3.2
τe_th	s	1.31	3.30
τe_ds03	s	1.05	2.50
ds03	-	1.24	1.32

-------------------
P0_ini = 2.50 MPa
-------------------
Param	Un